<a href="https://colab.research.google.com/github/Amlan05/EquityMeanReversion/blob/main/2HR__Strategy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [267]:
pip install pandas-ta

In [268]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import numpy as np

In [269]:
file_path = '/content/RELI 2HR.csv'

In [270]:
data = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)

In [271]:
data.rename(columns={'close': 'Close', 'high': 'High', 'open': 'Open', 'low': 'Low'}, inplace=True)

In [272]:
def convert_volume(volume):
    if isinstance(volume, str):
        if 'K' in volume:
            return float(volume.replace('K', '').replace(',', '')) * 1_000
        elif 'M' in volume:
            return float(volume.replace('M', '').replace(',', '')) * 1_000_000
        else:
            return float(volume.replace(',', ''))
    return volume

# Apply the conversion
data['Volume'] = data['Volume'].apply(convert_volume)

In [273]:
data = data.sort_values(by='Date').reset_index(drop=True)

In [274]:
print(data.tail())

                           Date     Open     High      Low   Close   Volume
5019  2025-01-24T15:00:00+05:30  1246.10  1248.00  1243.50  1246.3  1776011
5020  2025-01-27T09:00:00+05:30  1239.10  1240.45  1220.00  1220.4  3317612
5021  2025-01-27T11:00:00+05:30  1220.50  1233.50  1220.00  1230.8  3039450
5022  2025-01-27T13:00:00+05:30  1230.80  1235.90  1225.00  1230.0  2343373
5023  2025-01-27T15:00:00+05:30  1230.15  1231.65  1227.85  1229.0   789840


In [275]:
# Calculate Support and Resistance
data['Support'] = data['Low'][data['Low'] == data['Low'].rolling(window=9, center=True).min()]
data['Resistance'] = data['High'][data['High'] == data['High'].rolling(window=9, center=True).max()]

# Fill missing values with the previous value (forward-fill)
data['Support'] = data['Support'].ffill()
data['Resistance'] = data['Resistance'].ffill()

In [276]:
# RSI with SMA
data['RSI'] = ta.rsi(data['Close'], length=14, mamode="sma")

In [277]:
# MACD
macd_data = ta.macd(data['Close'], fast=12, slow=26, signal=9, append=True)
data['MACD_Histogram'] = macd_data['MACDh_12_26_9']

In [278]:
# ATR
data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)

In [279]:
# Bollinger Bands
bollinger = ta.bbands(data['Close'], length=20, std=2)
data['Upper_BB'] = bollinger['BBU_20_2.0']
data['Lower_BB'] = bollinger['BBL_20_2.0']
data['Middle_BB'] = bollinger['BBM_20_2.0']

In [280]:
# Moving Averages
data['100_MA'] = ta.sma(data['Close'], length=100)
data['50_MA'] = ta.sma(data['Close'], length=50)
data['20_MA'] = ta.sma(data['Close'], length=20)

In [281]:
# Rolling Volume Mean
data['Rolling_Volume_Mean'] = data['Volume'].rolling(window=10).mean()

In [282]:
data.tail()

,Date,Open,High,Low,Close,Volume,Support,Resistance,RSI,MACD_Histogram,ATR,Upper_BB,Lower_BB,Middle_BB,100_MA,50_MA,20_MA,Rolling_Volume_Mean
5019,2025-01-24T15:00:00+05:30,1246.10,1248.00,1243.50,1246.3,1776011,1227.25,1326.0,33.954053,-4.929693,10.807905,1312.723006,1239.111994,1275.9175,1245.8535,1264.042,1275.9175,2735702.2
5020,2025-01-27T09:00:00+05:30,1239.10,1240.45,1220.00,1220.4,3317612,1227.25,1326.0,24.301942,-6.311105,11.914483,1312.209682,1230.685318,1271.4475,1245.7135,1263.120,1271.4475,2833142.0
5021,2025-01-27T11:00:00+05:30,1220.50,1233.50,1220.00,1230.8,3039450,1227.25,1326.0,32.588649,-6.166885,12.027734,1309.614363,1226.085637,1267.8500,1245.8070,1262.426,1267.8500,2922906.3
5022,2025-01-27T13:00:00+05:30,1230.80,1235.90,1225.00,1230.0,2343373,1227.25,1326.0,32.295773,-5.774692,11.947182,1305.061486,1223.038514,1264.0500,1245.9870,1261.975,1264.0500,2991081.7
5023,2025-01-27T15:00:00+05:30,1230.15,1231.65,1227.85,1229.0,789840,1227.25,1326.0,31.909730,-5.247235,11.365240,1299.697868,1220.902132,1260.3000,1246.2240,1261.473,1260.3000,2789937.4


In [283]:
# Price Action Logic
def detect_price_action(row):
    body = abs(row['Close'] - row['Open'])
    wick_up = row['High'] - max(row['Close'], row['Open'])
    wick_down = min(row['Close'], row['Open']) - row['Low']

    if wick_up > 2 * body and wick_down < body and row['Close'] < row['Open']:
        return 'Shooting Star'
    elif wick_down > 2 * body and wick_up < body and row['Close'] > row['Open']:
        return 'Hammer'
    elif body > wick_up + wick_down and row['Volume'] > row['Rolling_Volume_Mean']:
        if row['Close'] < row['Open']:
            return 'Large Body Down'
        elif row['Close'] > row['Open']:
            return 'Large Body Up'
    return None

data['Price_Action'] = data.apply(detect_price_action, axis=1)
data.tail()

,Date,Open,High,Low,Close,Volume,Support,Resistance,RSI,MACD_Histogram,ATR,Upper_BB,Lower_BB,Middle_BB,100_MA,50_MA,20_MA,Rolling_Volume_Mean,Price_Action
5019,2025-01-24T15:00:00+05:30,1246.10,1248.00,1243.50,1246.3,1776011,1227.25,1326.0,33.954053,-4.929693,10.807905,1312.723006,1239.111994,1275.9175,1245.8535,1264.042,1275.9175,2735702.2,None
5020,2025-01-27T09:00:00+05:30,1239.10,1240.45,1220.00,1220.4,3317612,1227.25,1326.0,24.301942,-6.311105,11.914483,1312.209682,1230.685318,1271.4475,1245.7135,1263.120,1271.4475,2833142.0,Large Body Down
5021,2025-01-27T11:00:00+05:30,1220.50,1233.50,1220.00,1230.8,3039450,1227.25,1326.0,32.588649,-6.166885,12.027734,1309.614363,1226.085637,1267.8500,1245.8070,1262.426,1267.8500,2922906.3,Large Body Up
5022,2025-01-27T13:00:00+05:30,1230.80,1235.90,1225.00,1230.0,2343373,1227.25,1326.0,32.295773,-5.774692,11.947182,1305.061486,1223.038514,1264.0500,1245.9870,1261.975,1264.0500,2991081.7,None
5023,2025-01-27T15:00:00+05:30,1230.15,1231.65,1227.85,1229.0,789840,1227.25,1326.0,31.909730,-5.247235,11.365240,1299.697868,1220.902132,1260.3000,1246.2240,1261.473,1260.3000,2789937.4,None


In [284]:
# Backtesting Logic
position = 0
entry_price = None
stop_loss_price = None
above_ma_volume_counter = 0
exit_candle = 0
data['Buy_Date'] = None
data['Sell_Date'] = None
data['Stop_Loss'] = None

In [285]:
# def find_entry(data, start_index, position=0):
#     for index in range(start_index, len(data)):

#         row = data.iloc[index]
#         prevRow = data.iloc[index - 1]

#         # Long Condition 1
#         if (
#             position == 0
#             and row['RSI'] <= 40
#             and row['MACD_Histogram'] > prevRow['MACD_Histogram']
#             # and prevRow['Open'] < prevRow['Lower_BB'] and prevRow['Close'] > prevRow['Lower_BB']
#             and prevRow['Close'] > row['20_MA']
#         ):
#             entry_price = row['Open']
#             entry_date = row['Date']
#             stop_loss_price = min(entry_price - 2 * row['ATR'], row['Support'])
#             return index, entry_date, entry_price, stop_loss_price, 1


#         if (
#             position == 0
#             and row['RSI'] <= 50
#             and row['MACD_Histogram'] > prevRow['MACD_Histogram']
#             and prevRow['Open'] < prevRow['Lower_BB'] and prevRow['Close'] > prevRow['Lower_BB']
#             and row['Open'] > row['Support']
#         ):
#             entry_price = row['Open']
#             entry_date = row['Date']
#             stop_loss_price = min(entry_price - 2 * row['ATR'], row['Support'])
#             return index, entry_date, entry_price, stop_loss_price, 1

#         # Long Condition 2 (Hammer Reversal)
#         if (
#             position == 0
#             and prevRow['Price_Action'] == 'Hammer'
#             and row['Volume'] > row['Rolling_Volume_Mean']
#             and row['RSI'] <= 50
#             and row['Close'] > 1.01 * row['Lower_BB']

#         ):
#             entry_price = row['Close']
#             entry_date = row['Date']
#             stop_loss_price = min(entry_price - 2 * row['ATR'], row['Support'])
#             return index, entry_date, entry_price, stop_loss_price, 1

#         # Large Body Up
#         if (
#             position == 0
#             and prevRow['Price_Action'] == 'Large Candle Up'
#             # and row['Open'] < row['Close']
#             and row['Volume'] > row['Rolling_Volume_Mean']
#             and row['RSI'] <= 50
#             and row['Close'] > 1.01 * row['Lower_BB']
#         ):
#             entry_price = row['Open']
#             entry_date = row['Date']
#             stop_loss_price = min(entry_price - 2 * row['ATR'], row['Support'])
#             return index, entry_date, entry_price, stop_loss_price, 1

#         # Short Condition 1 (existing)
#         if (
#             position == 0
#             and row['RSI'] >= 60
#             and row['MACD_Histogram'] < prevRow['MACD_Histogram']
#             # and prevRow['Close'] < prevRow['Upper_BB']
#             and prevRow['Close'] < row['20_MA']
#             # and row['Open'] < row['Resistance']
#         ):
#             entry_price = row['Open']
#             entry_date = row['Date']
#             stop_loss_price = max(entry_price + 2 * row['ATR'], row['Resistance'])
#             return index, entry_date, entry_price, stop_loss_price, -1

#         if (
#             position == 0
#             and row['RSI'] >= 60
#             and row['MACD_Histogram'] < prevRow['MACD_Histogram']
#             and prevRow['Close'] < prevRow['Upper_BB']
#             # and row['Open'] < row['20_MA']
#             and row['Open'] < row['Resistance']
#         ):
#             entry_price = row['Open']
#             entry_date = row['Date']
#             stop_loss_price = max(entry_price + 2 * row['ATR'], row['Resistance'])
#             return index, entry_date, entry_price, stop_loss_price, -1

#         # Short Condition 2 (Shooting Star Reversal)
#         if (
#             position == 0
#             and prevRow['Price_Action'] == 'Shooting Star'
#             # and row['Close'] < row['Open']
#             and row['Volume'] > row['Rolling_Volume_Mean']
#             and row['RSI'] >= 50
#             and row['Open'] <= 0.99 * row['Upper_BB']
#         ):
#             entry_price = row['Close']
#             entry_date = row['Date']
#             stop_loss_price = max(entry_price + 2 * row['ATR'], row['Resistance'])
#             return index, entry_date, entry_price, stop_loss_price, -1

#         # Large Body Down
#         if (
#             position == 0
#             and prevRow['Price_Action'] == 'Large Candle Down'
#             # and row['Open'] > row['Close']
#             and row['Volume'] > row['Rolling_Volume_Mean']
#             and row['RSI'] >= 50
#             and row['Open'] <= 0.99 * row['Upper_BB']
#         ):
#             entry_price = row['Open']
#             entry_date = row['Date']
#             stop_loss_price = max(entry_price + 2 * row['ATR'], row['Resistance'])
#             return index, entry_date, entry_price, stop_loss_price, -1

#     # If no entry signal found
#     return None, None, None, None, 0


In [286]:
def find_entry(data, start_index, position=0):
    for index in range(start_index, len(data)):
        row = data.iloc[index]
        prevRow = data.iloc[index - 1]


        # Long Conditions
        # Long Condition 1: RSI, MACD, Moving Average, Volume, and Trend Filter
        if (
            position == 0
            and row['RSI'] >= 50
            and row['MACD_Histogram'] > prevRow['MACD_Histogram']
            and row['High'] > prevRow['High']
            and row['Close'] > prevRow['Support']
            and row['Close'] > row['20_MA']
            and row['Volume'] > prevRow['Volume']
            and row['Close'] < row['100_MA']
        ):
            entry_price = row['Open']
            entry_date = row['Date']
            stop_loss_price = min(entry_price - 1.5 * row['ATR'], row['Support'])
            return index, entry_date, entry_price, stop_loss_price, 1, 1

        # Long Condition 2: Hammer Reversal
        if (
            position == 0
            and prevRow['Price_Action'] == 'Hammer'
            and prevRow['Volume'] > prevRow['Rolling_Volume_Mean']
            and row['Close'] < prevRow['Close']
            and row['MACD_Histogram'] > prevRow['MACD_Histogram']
            and row['RSI'] <= 40
        ):
            entry_price = row['Close']
            entry_date = row['Date']
            stop_loss_price = min(entry_price - 1.5 * row['ATR'], row['Support'])
            return index, entry_date, entry_price, stop_loss_price, 1, 2

        # Long Condition 3: Large Candle Up
        if (
            position == 0
            and prevRow['Price_Action'] == 'Large Body Up'
            and row['Close'] > prevRow['Close']
            and row['MACD_Histogram'] > prevRow['MACD_Histogram']
            and row['RSI'] <= 40
        ):
            entry_price = row['Close']
            entry_date = row['Date']
            stop_loss_price = min(entry_price - 1.5 * row['ATR'], row['Support'])
            return index, entry_date, entry_price, stop_loss_price, 1, 3

        # Short Conditions
        # Short Condition 1: RSI, MACD, Moving Average, Volume, and Trend Filter
        if (
            position == 0
            and row['RSI'] <= 50
            and row['MACD_Histogram'] < prevRow['MACD_Histogram']
            and row['Low'] < prevRow['Low']
            and row['Close'] < prevRow['Resistance']
            and row['Close'] < row['20_MA']
            and row['Volume'] > prevRow['Volume']
            and row['Close'] > row['100_MA']
        ):
            entry_price = row['Open']
            entry_date = row['Date']
            stop_loss_price = max(entry_price + 1.5 * row['ATR'], row['Resistance'])
            return index, entry_date, entry_price, stop_loss_price, -1, 4

        # Short Condition 2: Shooting Star Reversal
        if (
            position == 0
            and prevRow['Price_Action'] == 'Shooting Star'
            and prevRow['Volume'] > prevRow['Rolling_Volume_Mean']
            and row['MACD_Histogram'] < prevRow['MACD_Histogram']
            and row['RSI'] >= 60
            and row['Close'] < prevRow['Close']
        ):
            entry_price = row['Close']
            entry_date = row['Date']
            stop_loss_price = max(entry_price + 1.5 * row['ATR'], row['Resistance'])
            return index, entry_date, entry_price, stop_loss_price, -1, 5

        # Short Condition 3: Large Candle Down
        if (
            position == 0
            and prevRow['Price_Action'] == 'Large Body Down'
            and row['MACD_Histogram'] < prevRow['MACD_Histogram']
            and row['RSI'] >= 60
            and row['Close'] < prevRow['Close']
        ):
            entry_price = row['Open']
            entry_date = row['Date']
            stop_loss_price = max(entry_price + 1.5 * row['ATR'], row['Resistance'])
            return index, entry_date, entry_price, stop_loss_price, -1, 6

    # If no entry signal found
    return None, None, None, None, 0, 0

In [287]:
def find_exit(data, entry_index, entry_price, stop_loss_price, position):
    support = float('nan')
    resistance = float('nan')
    above_ma_volume_counter = 0
    exit_candle = 0

    for index in range(entry_index + 1, len(data)):
        row = data.iloc[index]
        prevRow = data.iloc[index - 1] if index > 0 else None

        # Update support and resistance levels
        if prevRow is not None:
            if pd.notna(prevRow['Support']):
                support = prevRow['Support']
            if pd.notna(prevRow['Resistance']):
                resistance = prevRow['Resistance']

        # Exit logic for Buy position
        if position == 1:
            # Stop loss hit
            if row['Close'] <= stop_loss_price:
                return index, row['Date'], row['Close']

            # Case 1: Rejection at resistance or 100 MA
            # elif (row['High'] >= prevRow['Resistance'] and row['Close'] < prevRow['Resistance']) or \
            #      (row['High'] >= row['100_MA'] and row['Close'] <= row['100_MA']):
            #     return index, row['Date'], row['Close']


            elif (row['High'] >= prevRow['Resistance'] or row['High'] >= row['100_MA']):
              return index, row['Date'], row['Close']

            # Case 2: High volume red candles
            # elif (row['Open'] > row['100_MA'] or row['Open'] > resistance) and \
            #      (row['Close'] < row['100_MA'] or row['Close'] < resistance):
            #     if row['Volume'] > row['Rolling_Volume_Mean']:
            #         above_ma_volume_counter += 1
            #     elif row['Volume'] <= row['Rolling_Volume_Mean']:
            #         exit_candle += 1
            #     else:
            #       above_ma_volume_counter = 0
            #       exit_candle = 0

            # # # Exit after consecutive high volume candles
            # elif above_ma_volume_counter >= 1 and (row['Open'] < row['100_MA'] or row['Open'] < resistance):
            #     return index, row['Date'], row['Open']

            # # # Case 3: Low volume green candles
            # elif (row['Open'] < row['100_MA'] or row['Open'] < resistance) and exit_candle >= 1:
            #     return index, row['Date'], row['Close']



        # Exit logic for Sell position
        if position == -1:
            # Stop loss hit
            if row['Close'] >= stop_loss_price:
                return index, row['Date'], row['Close']

            # Case 1: Rejection at support or 100 MA
            # elif (row['Low'] <= support and row['Close'] > support) or \
            #      (row['Low'] < row['100_MA'] and row['Close'] > row['100_MA']):
            #     return index, row['Date'], row['Close']

            elif (row['Low'] <= prevRow['Support'] or row['Low'] <= row['100_MA']):
              return index, row['Date'], row['Close']

            # #Case 2: High volume green candles
            # elif (row['Open'] < row['100_MA'] or row['Open'] < support) and \
            #      (row['Close'] > row['100_MA'] or row['Close'] > support):
            #     if row['Volume'] > row['Rolling_Volume_Mean']:
            #         above_ma_volume_counter += 1
            #     elif row['Volume'] <= row['Rolling_Volume_Mean']:
            #         exit_candle += 1

            # # # Exit after consecutive high volume candles
            # elif above_ma_volume_counter >= 1 and (row['Open'] > row['100_MA'] or row['Open'] > support):
            #     return index, row['Date'], row['Open']

            # # # Case 3: Low volume green candles
            # elif (row['Open'] > row['100_MA'] or row['Open'] > support) and exit_candle >= 1:
            #     return index, row['Date'], row['Close']

    # If no exit is found within the dataset
    return None, None, None

In [288]:
# Main loop to calculate PnL and track trades
def calculate_pnl(data):
    position = 0
    start_index = 10
    total_pnl = 0
    num_trades = 0
    profit_trades = 0

    while start_index < len(data):
        # Find entry
        entry_index, entry_date, entry_price, stop_loss_price, position, entry_cond = find_entry(data, start_index, position)
        if entry_index is None:  # No more entries
            break

        print(f"Entry: {entry_date}, Price: {entry_price}, Entry_type: {position}, Entry_Cond: {entry_cond}")

        # Find exit
        exit_index, exit_date, exit_price = find_exit(data, entry_index, entry_price, stop_loss_price, position)
        if exit_index is None:  # No exit found
            break

        # Calculate PnL for the trade
        if position == 1:  # Long Position
           pnl = exit_price - entry_price
        elif position == -1:  # Short Position
           pnl = entry_price - exit_price

        if(pnl > 0):
            profit_trades+=1
        total_pnl += pnl
        num_trades += 1

        print(f"Exit: {exit_date}, Price: {exit_price}, PnL: {pnl}")

        start_index = exit_index + 1
        position = 0

    print(f"Total PnL: {total_pnl}, Number of Trades: {num_trades}, Successful Trades: {profit_trades}")
    return total_pnl, num_trades

# Run the calculation
calculate_pnl(data)

Entry: 2020-02-05T09:00:00+05:30, Price: 645.203225, Entry_type: 1, Entry_Cond: 1
Exit: 2020-02-12T09:00:00+05:30, Price: 657.8383985, PnL: 12.635173500000064
Entry: 2020-02-12T15:00:00+05:30, Price: 658.985007, Entry_type: 1, Entry_Cond: 1
Exit: 2020-02-13T09:00:00+05:30, Price: 658.737699, PnL: -0.24730799999997544
Entry: 2020-03-11T11:00:00+05:30, Price: 520.132992, Entry_type: 1, Entry_Cond: 3
Exit: 2020-03-12T09:00:00+05:30, Price: 482.0700935, PnL: -38.06289849999996
Entry: 2020-03-13T13:00:00+05:30, Price: 494.480442, Entry_type: 1, Entry_Cond: 3
Exit: 2020-03-19T09:00:00+05:30, Price: 405.382234, PnL: -89.098208
Entry: 2020-03-25T09:00:00+05:30, Price: 433.3729655, Entry_type: 1, Entry_Cond: 1
Exit: 2020-03-25T11:00:00+05:30, Price: 463.229746, PnL: 29.856780499999957
Entry: 2020-03-25T13:00:00+05:30, Price: 463.229746, Entry_type: 1, Entry_Cond: 1
Exit: 2020-04-03T09:00:00+05:30, Price: 476.3820175, PnL: 13.15227150000004
Entry: 2020-04-03T13:00:00+05:30, Price: 478.427926, En

(1262.8712075, 209)

In [289]:
data.iloc[231]

,231
Date,2020-03-25T09:00:00+05:30
Open,433.372966
High,464.983382
Low,433.125658
Close,451.74118
Volume,22338040
Support,393.736291
Resistance,465.343102
RSI,53.748966
MACD_Histogram,5.791848


In [290]:
data.head()

,Date,Open,High,Low,Close,Volume,Support,Resistance,RSI,MACD_Histogram,...,Lower_BB,Middle_BB,100_MA,50_MA,20_MA,Rolling_Volume_Mean,Price_Action,Buy_Date,Sell_Date,Stop_Loss
0,2020-01-02T11:00:00+05:30,687.223046,691.562171,685.896578,690.190738,5276040,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,2020-01-02T13:00:00+05:30,690.190738,692.888640,689.313920,690.527976,4800708,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2,2020-01-02T15:00:00+05:30,690.550458,691.472241,689.313920,690.348115,2412036,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
3,2020-01-03T09:00:00+05:30,689.313920,691.899410,684.817417,689.021647,10272348,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,2020-01-03T11:00:00+05:30,689.089095,692.281612,688.504550,691.831962,4614864,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
